# 🔄 บทที่ 2.3: Data Transformation - การแปลงข้อมูล

## 🎯 เรื่องราวของการแข่งวิ่ง

จินตนาการว่าคุณมีนักวิ่ง 3 คน:

- 👨 **คนที่ 1:** วิ่งได้ **5 km/hr** (เดินเล่นๆ)
- 👨 **คนที่ 2:** วิ่งได้ **15 km/hr** (วิ่งปกติ)
- 🏃 **คนที่ 3:** วิ่งได้ **50,000 steps/hr** (นับก้าว)

**ปัญหา:** คนที่ 3 ใช้หน่วยต่าง → ตัวเลขใหญ่กว่ามาก!

ถ้าเทรนโมเดล ML:
- โมเดลจะคิดว่าคนที่ 3 **สำคัญที่สุด** (เพราะตัวเลขใหญ่)
- ทั้งที่จริงๆ ทั้ง 3 คนวิ่งเร็วพอๆ กัน!

**วิธีแก้:** **Normalization** หรือ **Standardization**

มาเรียนรู้กันเลย! 🚀

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✅ Libraries loaded!")

---

## 🎭 Part 1: ทำไมต้อง Transform?

### ปัญหาของข้อมูลที่ไม่ได้ transform:

In [ ]:
# สร้างข้อมูลตัวอย่าง: Employee Data
np.random.seed(42)

data = {
    'Age': np.random.randint(22, 60, 200),
    'Salary': np.random.randint(25000, 150000, 200),
    'YearsExperience': np.random.randint(0, 30, 200),
    'ProjectsCompleted': np.random.randint(1, 100, 200)
}

df = pd.DataFrame(data)

print("👥 Employee Data (200 employees)")
print("="*60)
print(df.head(10))
print("\n📊 Statistical Summary:")
print(df.describe())

In [ ]:
# ปัญหา: Scale ต่างกันมาก!
print("⚠️ PROBLEM: Different Scales!\n")
print("Range of each feature:")
print("="*60)

for col in df.columns:
    min_val = df[col].min()
    max_val = df[col].max()
    range_val = max_val - min_val
    print(f"{col:20s}: [{min_val:6.0f}, {max_val:6.0f}]  Range: {range_val:6.0f}")

print("\n💡 Salary มีค่าใหญ่กว่า Age มาก!")
print("   → ML model จะให้ความสำคัญกับ Salary มากกว่า")
print("   → ทำให้การเรียนรู้ไม่สมดุล! ❌")

In [ ]:
# Visualize: เห็นชัดว่า scale ต่างกันมาก
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

for idx, col in enumerate(df.columns):
    row = idx // 2
    col_idx = idx % 2
    
    axes[row, col_idx].hist(df[col], bins=30, color='skyblue', edgecolor='black', alpha=0.7)
    axes[row, col_idx].set_title(f'{col} Distribution (Original)', fontsize=12, fontweight='bold')
    axes[row, col_idx].set_xlabel(col)
    axes[row, col_idx].set_ylabel('Frequency')
    axes[row, col_idx].axvline(df[col].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df[col].mean():.0f}')
    axes[row, col_idx].legend()
    axes[row, col_idx].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("👀 สังเกต: แกน X ของแต่ละ feature ต่างกันมาก!")

---

## 📏 Part 2: Min-Max Normalization (Scaling to 0-1)

### สูตร:
```
X_normalized = (X - X_min) / (X_max - X_min)
```

### ผลลัพธ์:
- ทุกค่าจะอยู่ระหว่าง **0 ถึง 1**
- ค่าน้อยสุด → 0
- ค่ามากสุด → 1

### เหมาะกับ:
✅ Neural Networks  
✅ Image data (pixel 0-255 → 0-1)  
✅ ข้อมูลที่มี bounded range

### ไม่เหมาะกับ:
❌ ข้อมูลที่มี outliers เยอะ (outliers จะดึง scale)

In [ ]:
# วิธีที่ 1: Manual calculation
print("🔧 Method 1: Manual Min-Max Normalization\n")

age_normalized_manual = (df['Age'] - df['Age'].min()) / (df['Age'].max() - df['Age'].min())

print(f"Age (Original):")
print(f"   Min: {df['Age'].min()}")
print(f"   Max: {df['Age'].max()}")
print(f"   Sample values: {df['Age'].head().values}")

print(f"\nAge (Normalized):")
print(f"   Min: {age_normalized_manual.min():.4f}")
print(f"   Max: {age_normalized_manual.max():.4f}")
print(f"   Sample values: {age_normalized_manual.head().values}")

print("\n✅ All values are now between 0 and 1!")

In [ ]:
# วิธีที่ 2: Using scikit-learn (แนะนำ!)
print("🛠️ Method 2: Using MinMaxScaler (Recommended)\n")

scaler = MinMaxScaler()
df_normalized = pd.DataFrame(
    scaler.fit_transform(df),
    columns=df.columns
)

print("Original Data:")
print(df.head())

print("\nNormalized Data (0-1):")
print(df_normalized.head())

print("\n📊 Normalized Statistics:")
print(df_normalized.describe())

In [ ]:
# Visualization: Before vs After
fig, axes = plt.subplots(2, 4, figsize=(20, 10))

for idx, col in enumerate(df.columns):
    # Before (top row)
    axes[0, idx].hist(df[col], bins=30, color='red', alpha=0.6, edgecolor='black')
    axes[0, idx].set_title(f'{col}\nBEFORE Normalization', fontsize=11, fontweight='bold')
    axes[0, idx].set_xlabel('Value')
    axes[0, idx].set_ylabel('Frequency')
    axes[0, idx].grid(alpha=0.3)
    
    # After (bottom row)
    axes[1, idx].hist(df_normalized[col], bins=30, color='green', alpha=0.6, edgecolor='black')
    axes[1, idx].set_title(f'{col}\nAFTER Normalization', fontsize=11, fontweight='bold')
    axes[1, idx].set_xlabel('Value (0-1)')
    axes[1, idx].set_ylabel('Frequency')
    axes[1, idx].set_xlim([-0.1, 1.1])
    axes[1, idx].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("✨ ตอนนี้ทุก feature อยู่ใน scale เดียวกัน (0-1)!")

---

## 📊 Part 3: Standardization (Z-Score Normalization)

### สูตร:
```
X_standardized = (X - mean) / std
```

### ผลลัพธ์:
- Mean = **0**
- Standard Deviation = **1**
- ค่าส่วนใหญ่จะอยู่ระหว่าง **-3 ถึง +3**

### เหมาะกับ:
✅ Linear Regression  
✅ SVM  
✅ ข้อมูลที่มี Gaussian distribution  
✅ อัลกอริทึมที่ใช้ distance (KNN, K-Means)

### ข้อดี:
✅ ไม่ถูก bounded (ไม่จำกัดแค่ 0-1)  
✅ Preserve outliers information

In [ ]:
# วิธีที่ 1: Manual calculation
print("🔧 Method 1: Manual Standardization\n")

age_standardized_manual = (df['Age'] - df['Age'].mean()) / df['Age'].std()

print(f"Age (Original):")
print(f"   Mean: {df['Age'].mean():.2f}")
print(f"   Std: {df['Age'].std():.2f}")
print(f"   Sample: {df['Age'].head().values}")

print(f"\nAge (Standardized):")
print(f"   Mean: {age_standardized_manual.mean():.10f} (≈ 0)")
print(f"   Std: {age_standardized_manual.std():.10f} (≈ 1)")
print(f"   Sample: {age_standardized_manual.head().values}")

print("\n✅ Mean ≈ 0, Std ≈ 1")

In [ ]:
# วิธีที่ 2: Using StandardScaler (แนะนำ!)
print("🛠️ Method 2: Using StandardScaler\n")

scaler_std = StandardScaler()
df_standardized = pd.DataFrame(
    scaler_std.fit_transform(df),
    columns=df.columns
)

print("Original Data:")
print(df.head())

print("\nStandardized Data (mean=0, std=1):")
print(df_standardized.head())

print("\n📊 Standardized Statistics:")
print(df_standardized.describe())

In [ ]:
# Visualization: Before vs After
fig, axes = plt.subplots(2, 4, figsize=(20, 10))

for idx, col in enumerate(df.columns):
    # Before (top row)
    axes[0, idx].hist(df[col], bins=30, color='orange', alpha=0.6, edgecolor='black')
    axes[0, idx].set_title(f'{col}\nBEFORE Standardization', fontsize=11, fontweight='bold')
    axes[0, idx].axvline(df[col].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df[col].mean():.0f}')
    axes[0, idx].set_xlabel('Value')
    axes[0, idx].set_ylabel('Frequency')
    axes[0, idx].legend()
    axes[0, idx].grid(alpha=0.3)
    
    # After (bottom row)
    axes[1, idx].hist(df_standardized[col], bins=30, color='blue', alpha=0.6, edgecolor='black')
    axes[1, idx].set_title(f'{col}\nAFTER Standardization', fontsize=11, fontweight='bold')
    axes[1, idx].axvline(0, color='red', linestyle='--', linewidth=2, label='Mean: 0')
    axes[1, idx].set_xlabel('Z-Score')
    axes[1, idx].set_ylabel('Frequency')
    axes[1, idx].legend()
    axes[1, idx].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("✨ ทุก feature มี mean=0 และ std=1 แล้ว!")

---

## 🎯 Part 4: Min-Max vs Standardization - เปรียบเทียบ

มาดูกันว่าแตกต่างกันอย่างไร!

In [ ]:
# เปรียบเทียบ side-by-side
comparison_df = pd.DataFrame({
    'Original_Age': df['Age'].head(10).values,
    'Normalized_Age': df_normalized['Age'].head(10).values,
    'Standardized_Age': df_standardized['Age'].head(10).values,
})

print("📊 Comparison: Original vs Normalized vs Standardized")
print("="*70)
print(comparison_df)

print("\n💡 สังเกต:")
print("   - Normalized: อยู่ระหว่าง 0-1")
print("   - Standardized: อาจเป็นลบหรือบวกได้, ส่วนใหญ่อยู่ระหว่าง -3 ถึง +3")

In [ ]:
# Visualization: เปรียบเทียบทั้ง 3 แบบ
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Original
axes[0].hist(df['Salary'], bins=30, color='gray', alpha=0.7, edgecolor='black')
axes[0].set_title('Original Salary\n(25,000 - 150,000)', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Salary ($)')
axes[0].set_ylabel('Frequency')
axes[0].grid(alpha=0.3)

# Normalized
axes[1].hist(df_normalized['Salary'], bins=30, color='green', alpha=0.7, edgecolor='black')
axes[1].set_title('Min-Max Normalized\n(0 - 1)', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Normalized Salary')
axes[1].set_ylabel('Frequency')
axes[1].set_xlim([-0.1, 1.1])
axes[1].grid(alpha=0.3)

# Standardized
axes[2].hist(df_standardized['Salary'], bins=30, color='blue', alpha=0.7, edgecolor='black')
axes[2].set_title('Standardized\n(mean=0, std=1)', fontsize=13, fontweight='bold')
axes[2].set_xlabel('Z-Score')
axes[2].set_ylabel('Frequency')
axes[2].axvline(0, color='red', linestyle='--', linewidth=2, label='Mean=0')
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n👀 เห็นความแตกต่างไหม?")
print("   - Shape ของ distribution ยังเหมือนเดิม")
print("   - เปลี่ยนแค่ scale (แกน X)")

---

## 🛡️ Part 5: Robust Scaler (สำหรับข้อมูลที่มี Outliers)

### ปัญหาของ Standardization:
- ใช้ **mean** และ **std**
- Outliers ทำให้ mean และ std เพี้ยน!

### วิธีแก้: Robust Scaler
- ใช้ **median** และ **IQR** แทน
- ไม่ถูกผลกระทบจาก outliers

### สูตร:
```
X_robust = (X - median) / IQR
```

In [ ]:
# สร้างข้อมูลที่มี outliers
np.random.seed(42)

data_with_outliers = {
    'Score': np.concatenate([
        np.random.normal(50, 10, 180),  # ข้อมูลปกติ
        np.array([200, 250, 300, -50, -100] * 4)  # Outliers!
    ])
}

df_outliers = pd.DataFrame(data_with_outliers)

print("📊 Data with Outliers:")
print(f"   Mean: {df_outliers['Score'].mean():.2f}")
print(f"   Median: {df_outliers['Score'].median():.2f}")
print(f"   Std: {df_outliers['Score'].std():.2f}")
print(f"   Min: {df_outliers['Score'].min():.2f}")
print(f"   Max: {df_outliers['Score'].max():.2f}")

# Visualize
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.hist(df_outliers['Score'], bins=50, color='coral', edgecolor='black', alpha=0.7)
plt.title('Score Distribution (With Outliers)', fontsize=13, fontweight='bold')
plt.xlabel('Score')
plt.ylabel('Frequency')
plt.axvline(df_outliers['Score'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df_outliers["Score"].mean():.1f}')
plt.axvline(df_outliers['Score'].median(), color='green', linestyle='--', linewidth=2, label=f'Median: {df_outliers["Score"].median():.1f}')
plt.legend()
plt.grid(alpha=0.3)

plt.subplot(1, 2, 2)
plt.boxplot(df_outliers['Score'])
plt.title('Box Plot (Outliers Visible)', fontsize=13, fontweight='bold')
plt.ylabel('Score')
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# เปรียบเทียบ 3 วิธี กับข้อมูลที่มี outliers
scaler_minmax = MinMaxScaler()
scaler_standard = StandardScaler()
scaler_robust = RobustScaler()

score_normalized = scaler_minmax.fit_transform(df_outliers)
score_standardized = scaler_standard.fit_transform(df_outliers)
score_robust = scaler_robust.fit_transform(df_outliers)

# Visualization
fig, axes = plt.subplots(1, 4, figsize=(20, 5))

# Original
axes[0].hist(df_outliers['Score'], bins=50, color='gray', alpha=0.7, edgecolor='black')
axes[0].set_title('Original\n(With Outliers)', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Score')
axes[0].set_ylabel('Frequency')

# Min-Max
axes[1].hist(score_normalized, bins=50, color='green', alpha=0.7, edgecolor='black')
axes[1].set_title('Min-Max Normalized\n❌ Affected by outliers', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Normalized Score')
axes[1].set_ylabel('Frequency')

# Standard
axes[2].hist(score_standardized, bins=50, color='blue', alpha=0.7, edgecolor='black')
axes[2].set_title('Standardized\n❌ Affected by outliers', fontsize=12, fontweight='bold')
axes[2].set_xlabel('Z-Score')
axes[2].set_ylabel('Frequency')

# Robust
axes[3].hist(score_robust, bins=50, color='purple', alpha=0.7, edgecolor='black')
axes[3].set_title('Robust Scaler\n✅ NOT affected by outliers!', fontsize=12, fontweight='bold')
axes[3].set_xlabel('Robust Score')
axes[3].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

print("\n✅ Robust Scaler is BEST for data with outliers!")

---

## 📋 Part 6: สรุป - เลือกวิธีไหนดี?

### Decision Tree: 🌳

In [ ]:
# สร้างตารางสรุป
summary = pd.DataFrame({
    'Method': ['Min-Max Normalization', 'Standardization', 'Robust Scaler'],
    'Formula': [
        '(X - min) / (max - min)',
        '(X - mean) / std',
        '(X - median) / IQR'
    ],
    'Output Range': [
        '0 to 1',
        '-∞ to +∞ (mostly -3 to +3)',
        '-∞ to +∞'
    ],
    'Best For': [
        'Neural Networks, Images, Bounded data',
        'Linear models, SVM, Gaussian data',
        'Data with many outliers'
    ],
    'Sensitive to Outliers': [
        '❌ Yes (Very)',
        '❌ Yes (Moderately)',
        '✅ No (Robust!)'
    ]
})

print("📊 Scaling Methods Comparison")
print("="*100)
print(summary.to_string(index=False))

In [ ]:
# แนวทางการเลือก
print("\n🎯 How to Choose? Decision Guide:\n")
print("="*70)

print("\n1️⃣ ถ้าคุณใช้ NEURAL NETWORKS หรือ DEEP LEARNING:")
print("   → ใช้ Min-Max Normalization (0-1) ✅")
print("   → หรือ Standardization ก็ได้")

print("\n2️⃣ ถ้าข้อมูลมี OUTLIERS เยอะ:")
print("   → ใช้ Robust Scaler ✅")
print("   → หรือลบ outliers ก่อน แล้วใช้วิธีอื่น")

print("\n3️⃣ ถ้าใช้ LINEAR MODELS (Linear Regression, SVM):")
print("   → ใช้ Standardization ✅")

print("\n4️⃣ ถ้าข้อมูลเป็น IMAGE (pixels 0-255):")
print("   → แบ่ง 255.0 (เท่ากับ normalize to 0-1) ✅")

print("\n5️⃣ ถ้าใช้ TREE-BASED models (Random Forest, XGBoost):")
print("   → ไม่จำเป็นต้อง scale! Trees ไม่สนใจ scale 🌲")

print("\n💡 Golden Rule:")
print("   ทดลองหลายวิธี แล้วเลือกที่ให้ผลดีที่สุด!")

---

## 💾 Part 7: Save & Load Scalers

**สำคัญมาก!** ต้องเก็บ scaler ไว้ใช้กับ test data!

In [ ]:
import joblib

# บันทึก scaler
scaler = MinMaxScaler()
scaler.fit(df)

# Save
joblib.dump(scaler, 'minmax_scaler.pkl')
print("💾 Scaler saved to 'minmax_scaler.pkl'")

# Load
loaded_scaler = joblib.load('minmax_scaler.pkl')
print("📂 Scaler loaded successfully!")

# ใช้กับข้อมูลใหม่
new_data = np.array([[35, 75000, 8, 25]])  # อายุ 35, เงินเดือน 75000, ประสบการณ์ 8 ปี, 25 โปรเจกต์
new_data_scaled = loaded_scaler.transform(new_data)

print("\n✅ Transform new data:")
print(f"   Original: {new_data[0]}")
print(f"   Scaled: {new_data_scaled[0]}")

print("\n⚠️ Important: ต้องใช้ scaler ตัวเดียวกันกับที่ fit บน training data!")

---

## 📝 สรุปบทเรียน

### ✅ สิ่งที่เราได้เรียนรู้:

1. **ทำไมต้อง Transform?**
   - Features มี scale ต่างกัน → โมเดลเรียนรู้ไม่สมดุล
   - Scaling ทำให้ทุก feature มีความสำคัญเท่ากัน

2. **Min-Max Normalization**
   - Scale to [0, 1]
   - เหมาะกับ Neural Networks
   - ไวต่อ outliers

3. **Standardization**
   - Mean = 0, Std = 1
   - เหมาะกับ Linear models
   - Preserve outlier information

4. **Robust Scaler**
   - ใช้ median และ IQR
   - ทนต่อ outliers
   - แนะนำถ้าข้อมูลมี outliers เยอะ

5. **Best Practices**
   - Fit scaler บน training data เท่านั้น
   - Transform ทั้ง train และ test ด้วย scaler ตัวเดียวกัน
   - เก็บ scaler ไว้ใช้กับ production data

### 🚀 Next Step:

ต่อไปเราจะเรียนรู้:

👉 [Data Augmentation - เพิ่มข้อมูลภาพ](04_data_augmentation.ipynb)

เทคนิคสำคัญสำหรับ Deep Learning กับภาพ!